In [13]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import datetime
from datetime import timedelta, date

In [141]:
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates

d = SchoolHolidayDates()
ho_20 = d.holidays_for_year_and_zone(2020, 'A')
ho_21 = d.holidays_for_year_and_zone(2021, 'A')

jf_20 = JoursFeries.for_year(2020)
jf_21 = JoursFeries.for_year(2021)

In [131]:
df_ho20 = pd.DataFrame({'date': key, 'vacances': value['vacances_zone_a']} for key, value in ho_20.items())
df_ho21 = pd.DataFrame({'date': key, 'vacances': value['vacances_zone_a']} for key, value in ho_20.items())
df_ho = pd.concat([df_ho20,df_ho21]).reset_index()

df_ho.head()

,index,date,vacances
0,0,2020-01-01,True
1,1,2020-01-02,True
2,2,2020-01-03,True
3,3,2020-01-04,True
4,4,2020-01-05,True


In [132]:
df_jf_21 =pd.DataFrame(jf_21.keys(),jf_21.values(),columns=['fete']).reset_index().rename(columns={'index':'date'})
df_jf_20 =pd.DataFrame(jf_20.keys(),jf_20.values(),columns=['fete']).reset_index().rename(columns={'index':'date'})
df_jf = pd.concat([df_jf_20,df_jf_21]).reset_index()
df_jf.head()

,index,date,fete
0,0,2020-01-01,1er janvier
1,1,2020-04-13,Lundi de Pâques
2,2,2020-05-01,1er mai
3,3,2020-05-08,8 mai
4,4,2020-05-21,Ascension


In [133]:
start_date = date(2020, 9, 9)
end_date = date(2021, 9, 9)
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

df = pd.DataFrame({'date': date_range})

df

,date
0,2020-09-09
1,2020-09-10
2,2020-09-11
3,2020-09-12
4,2020-09-13
...,...
361,2021-09-05
362,2021-09-06
363,2021-09-07
364,2021-09-08


In [139]:
final = df.merge(df_jf, how='left', on='date').fillna(0)
final['is_jf'] = final['fete'].map(lambda x: 1 if x!=0 else 0)
final = final.merge(df_ho, how='left', on='date')
final['is_ho'] = final['vacances'].map(lambda x: 1 if x == True else 0)

columns_to_keep = ['date', 'is_ho', 'is_jf']
final = final[columns_to_keep]

final

,date,is_ho,is_jf
0,2020-09-09,0,0
1,2020-09-10,0,0
2,2020-09-11,0,0
3,2020-09-12,0,0
4,2020-09-13,0,0
...,...,...,...
390,2021-09-05,0,0
391,2021-09-06,0,0
392,2021-09-07,0,0
393,2021-09-08,0,0


In [140]:
csv_path = '/Users/henrivignial/Desktop/python_dsb_final_project/holidays_ferie.csv'
final.to_csv(csv_path, index = False)